## Reddit Hackathon

### Import Necessary dependencies here

In [1]:
import dnalab
import spacy 
import pandas as pd
import os

!pip3 install -U spacy
!python3 -m spacy download en_core_web_sm

     |████████████████████████████████| 13.7 MB 12.8 MB/s eta 0:00:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


### Use `en_core_web_sm` model from spacy for better accuracy

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
PROJECT_PATH = os.getcwd()
POSTS_PATH = 'data/reddit_posts'
COMMENTS_PATH = 'data/reddit_comments'
POSTS_DATA_PATH = os.path.join(PROJECT_PATH, POSTS_PATH)
COMMENTS_DATA_PATH = os.path.join(PROJECT_PATH, COMMENTS_PATH)

### Load CSV Files into a Pandas Dataframe

Go through all the files in the reddit_posts and reddit_comments directory and load them into a pandas df

In [4]:
def get_df(input_path, output_path):
    all_files = glob.glob(input_path + "/*.csv")

    dfs = []

    for filename in all_files:
        df = pd.read_csv(filename, index_col=None)
        dfs.append(df)

    df = pd.concat(dfs, axis=0, ignore_index=True)
    non_nan_df = df[df['text'].notna()]
    filtered_df = non_nan_df[(non_nan_df['text'] != "[removed]") & (
        non_nan_df['text'] != '[deleted]')]
    filtered_df['date'] = filtered_df['timestamp'].apply(
        lambda x: datetime.fromtimestamp(x)).tolist()
    filtered_df.set_index('date', inplace=True)
    final_df = filtered_df.drop(filtered_df.columns[0], axis=1)
    final_df = final_df.sort_index()
    final_df = final_df[final_df.index.month == 3]
    final_df.to_csv(output_path)
    

    
def get_input(data_path, column):
    df = pd.read_csv(data_path, index_col='date', parse_dates=True)
    res = df[column].tolist()
    return res

### Specify Blacklist

This should be extended to include more words according to Reddit posts/comments

In [5]:
BLACKLIST = ['bot', 'nyse', 'wsb', 'stock', 'mod', 'yolo', 'sec']

### Extract Symbols

In [6]:
def process(raw_comment, nlp):
    doc = nlp(raw_comment)
    tokens = []
    res = None
    for ent in doc.ents:
        if ent.text.lower() not in BLACKLIST and ent.label_ == "ORG":
            tokens.append(str(ent))
            
    if len(tokens) > 0:
        res = ", ".join(tokens)
    return res

In [7]:
posts_df = pd.read_csv(os.path.join(PROJECT_PATH, 'data/posts_df.csv'), parse_dates=True)
posts_df['type'] = 'P'

In [8]:
comments_df = pd.read_csv(os.path.join(PROJECT_PATH, 'data/comments_df.csv'), parse_dates=True)
comments_df['type'] = 'C'

In [9]:
posts_df.head(10)

,date,post_id,text,title,upvote_ratio,timestamp,author,type
0,2021-03-02 09:25:06,lw4h9d,"So loving this GME still going strong, that I ...",GME and the righteous options?,1.0,1614698706,Kick-Effective,P
1,2021-03-02 09:25:57,lw4i0b,Is this being pushed here. Been shareholder f...,Rocket Companies,1.0,1614698757,Master-Unit6692,P
2,2021-03-02 09:26:04,lw4i3o,"Gentlemen,\n\nNow that the yield bubble is pop...",Cathie Wood said PLTR is mandatory,1.0,1614698764,boomerbubblebuster,P
3,2021-03-02 09:26:29,lw4ihe,CVSI is on the move up! Buy low sell high! Hav...,CVSI,1.0,1614698789,BigShr1mp,P
4,2021-03-02 09:26:36,lw4ilh,There is probably some rule against this threa...,NOT the GME thread but share your thoughts here,1.0,1614698796,SwedishKillerChef,P
5,2021-03-02 09:26:47,lw4is2,Fk these mods,Gee Emm Eey 3/2/21,1.0,1614698807,Mosaikmuster,P
6,2021-03-02 09:27:06,lw4j12,Because apparently so,"RKT MEGATHREAD March 2, 2020",1.0,1614698826,iamzyb,P
7,2021-03-02 09:27:42,lw4jj8,🚀🚀🚀 BankOfSabadell 🚀🚀🚀🚀\n\n*Processing img 0b...,BankOfSanadell -- Spain,1.0,1614698862,NoFlySS,P
8,2021-03-02 09:27:57,lw4jqn,Mods surely know that no GME thread = spam in ...,Something fishy is going on?,1.0,1614698877,Crowbar_Freeman,P
9,2021-03-02 09:28:07,lw4juw,CBD Life Sciences Inc. is up today. It's a C...,CBDL Up,1.0,1614698887,ThisGuyAlright,P


In [10]:
comments_df.head(10)

,date,comment_id,text,score,awards,timestamp,author,\t\t,type
0,2021-03-02 09:27:05,gpf8jcq,Heavy resistance at 124! Needs to move through...,1,0,1614698825,jsams14,NaN,C
1,2021-03-02 09:27:25,gpf8ky5,Honestly mods why didn't you make an actual GM...,2,0,1614698845,SwedishKillerChef,NaN,C
2,2021-03-02 09:28:03,gpf8nvm,Squeeze,2,0,1614698883,elliskj1979,NaN,C
3,2021-03-02 09:28:07,gpf8o7q,And there she goes (:,2,0,1614698887,SARMY1K,NaN,C
4,2021-03-02 09:29:00,gpf8sfw,Boom you see how quick she ran after blasting ...,1,0,1614698940,jsams14,NaN,C
5,2021-03-02 09:29:38,gpf8vfy,No shit!\n\nhttps://fintel.io/ss/us/rkt,1,0,1614698978,robertleeblairjr,NaN,C
6,2021-03-02 09:29:40,gpf8vly,Let's just talk GME here,3,0,1614698980,HouseDowningVicodin,NaN,C
7,2021-03-02 09:29:45,gpf8vyp,Fr,1,0,1614698985,420yolotrillswag,NaN,C
8,2021-03-02 09:29:46,gpf8w2i,Holding RKT and GME. Suck on that Mods,1,0,1614698986,gswblu3-1lead,NaN,C
9,2021-03-02 09:29:55,gpf8wqh,RKT TO THE MOON!!! 🚀🚀🌚🌚,1,0,1614698995,Young0Ice,NaN,C


In [11]:
comments_df = comments_df[['comment_id', 'text', 'timestamp', 'type']]
posts_df = posts_df[['post_id', 'text', 'timestamp', 'type']]

In [12]:
comments_df = comments_df.loc[1:100, :]
posts_df = posts_df.loc[1:100, :]

In [13]:
comments_df['symbol'] = comments_df['text'].apply(lambda x: process(x, nlp))

In [14]:
comments_df.head(10)

,comment_id,text,timestamp,type,symbol
1,gpf8ky5,Honestly mods why didn't you make an actual GM...,1614698845,C,GME
2,gpf8nvm,Squeeze,1614698883,C,None
3,gpf8o7q,And there she goes (:,1614698887,C,None
4,gpf8sfw,Boom you see how quick she ran after blasting ...,1614698940,C,None
5,gpf8vfy,No shit!\n\nhttps://fintel.io/ss/us/rkt,1614698978,C,None
6,gpf8vly,Let's just talk GME here,1614698980,C,GME
7,gpf8vyp,Fr,1614698985,C,None
8,gpf8w2i,Holding RKT and GME. Suck on that Mods,1614698986,C,GME
9,gpf8wqh,RKT TO THE MOON!!! 🚀🚀🌚🌚,1614698995,C,🚀🚀🌚🌚
10,gpf8xim,It is coming for you,1614699005,C,None


In [15]:
posts_df['symbol'] = posts_df['text'].apply(lambda x: process(x, nlp))

In [16]:
posts_df.head(10)

,post_id,text,timestamp,type,symbol
1,lw4i0b,Is this being pushed here. Been shareholder f...,1614698757,P,None
2,lw4i3o,"Gentlemen,\n\nNow that the yield bubble is pop...",1614698764,P,PLTR
3,lw4ihe,CVSI is on the move up! Buy low sell high! Hav...,1614698789,P,CVSI
4,lw4ilh,There is probably some rule against this threa...,1614698796,P,GME
5,lw4is2,Fk these mods,1614698807,P,None
6,lw4j12,Because apparently so,1614698826,P,None
7,lw4jj8,🚀🚀🚀 BankOfSabadell 🚀🚀🚀🚀\n\n*Processing img 0b...,1614698862,P,None
8,lw4jqn,Mods surely know that no GME thread = spam in ...,1614698877,P,GME
9,lw4juw,CBD Life Sciences Inc. is up today. It's a C...,1614698887,P,"CBD Life Sciences Inc., Amazon"
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,"FDA, NPS, TAM, EV"


# beam_width : 

Number of alternate analyses to consider. More is slower, and not necessarily better -- you need to experiment on your problem. (by default : 1)

# beam_density : 

This clips solutions at each step. We multiply the score of the top-ranked action by this value, and use the result as a threshold. This prevents the parser from exploring options that look very unlikely, saving a bit of efficiency. Accuracy may also improve, because we've trained on greedy objective. (by default : 0)

In [17]:
from collections import defaultdict

def filter_low_confidence_entities(entities, cutoff=0.1):
    return {key: value for key, value in entities.items() if value > cutoff}

def get_momentum(text):
    doc = nlp.make_doc(text)
    beams = nlp.get_pipe("ner").beam_parse([doc], beam_width=1, beam_density=0.0000)
    entity_scores = defaultdict(float)
    total_score = 0
    for score, ents in nlp.get_pipe("ner").moves.get_beam_parses(beams[0]):
        total_score += score
        for start, end, label in ents:
            entity_scores[(start, end, label)] += score

    normalized_beam_score = {k: v/total_score for k, v in entity_scores.items()}
    confident_entities = filter_low_confidence_entities(normalized_beam_score)
    
    confident_entity_texts = {}
    
    for k, v in confident_entities.items():
        if k[2] == 'ORG':
            confident_entity_texts[str(doc[int(k[0]): int(k[1])])] = v*10

    return confident_entity_texts

In [18]:
comments_df = comments_df[comments_df['symbol'].notnull()]
comments_df['momentum_factor'] = comments_df['text'].apply(get_momentum)

In [19]:
posts_df = posts_df[posts_df['symbol'].notnull()]
posts_df['momentum_factor'] = posts_df['text'].apply(get_momentum)

In [20]:
comment_ticker_split = comments_df['symbol'].str.split(',').apply(pd.Series, 1).stack()
comment_ticker_split.index = comment_ticker_split.index.droplevel(-1)
comment_ticker_split.name = 'symbol'
del comments_df['symbol']
comment_ticker_split_df = comments_df.join(comment_ticker_split)

In [21]:
comment_ticker_split_df.head(10)

,comment_id,text,timestamp,type,momentum_factor,symbol
1,gpf8ky5,Honestly mods why didn't you make an actual GM...,1614698845,C,{'GME': 10.0},GME
6,gpf8vly,Let's just talk GME here,1614698980,C,{'GME': 10.0},GME
8,gpf8w2i,Holding RKT and GME. Suck on that Mods,1614698986,C,{'GME': 10.0},GME
9,gpf8wqh,RKT TO THE MOON!!! 🚀🚀🌚🌚,1614698995,C,{'🚀🚀🌚🌚': 10.0},🚀🚀🌚🌚
11,gpf8xou,I like GME,1614699007,C,{'GME': 10.0},GME
14,gpf8yoh,"This sub isn’t just a GME cesspool, jfc people",1614699021,C,{'GME': 10.0},GME
15,gpf8z1l,So apparently GME thread gets deleted in SECON...,1614699025,C,{'GME': 10.0},GME
19,gpf92y2,Very fishy... no zjz posts no Unpopular pinnn...,1614699076,C,"{'Unpopular': 10.0, 'GME': 10.0}",Unpopular
19,gpf92y2,Very fishy... no zjz posts no Unpopular pinnn...,1614699076,C,"{'Unpopular': 10.0, 'GME': 10.0}",GME
22,gpf93e3,![img](emote|wsb_emojis|bull) on CCIV / PLTR (...,1614699081,C,{'CCIV / PLTR (Aside': 10.0},CCIV / PLTR (


In [22]:
post_ticker_split = posts_df['symbol'].str.split(',').apply(pd.Series, 1).stack()
post_ticker_split.index = post_ticker_split.index.droplevel(-1)
post_ticker_split.name = 'symbol'
del posts_df['symbol']
post_ticker_split_df = posts_df.join(post_ticker_split)

In [23]:
post_ticker_split_df.head(10)

,post_id,text,timestamp,type,momentum_factor,symbol
2,lw4i3o,"Gentlemen,\n\nNow that the yield bubble is pop...",1614698764,P,{'PLTR': 10.0},PLTR
3,lw4ihe,CVSI is on the move up! Buy low sell high! Hav...,1614698789,P,{'CVSI': 10.0},CVSI
4,lw4ilh,There is probably some rule against this threa...,1614698796,P,{'GME': 10.0},GME
8,lw4jqn,Mods surely know that no GME thread = spam in ...,1614698877,P,{'GME': 10.0},GME
9,lw4juw,CBD Life Sciences Inc. is up today. It's a C...,1614698887,P,"{'CBD Life Sciences Inc.': 10.0, 'Amazon': 10.0}",CBD Life Sciences Inc.
9,lw4juw,CBD Life Sciences Inc. is up today. It's a C...,1614698887,P,"{'CBD Life Sciences Inc.': 10.0, 'Amazon': 10.0}",Amazon
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,"{'FDA': 10.0, 'NPS': 10.0, 'TAM': 10.0, 'EV': ...",FDA
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,"{'FDA': 10.0, 'NPS': 10.0, 'TAM': 10.0, 'EV': ...",NPS
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,"{'FDA': 10.0, 'NPS': 10.0, 'TAM': 10.0, 'EV': ...",TAM
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,"{'FDA': 10.0, 'NPS': 10.0, 'TAM': 10.0, 'EV': ...",EV


In [24]:
def extract_momentum(momentum_dict, ticker):
    if ticker in momentum_dict:
        result = momentum_dict[ticker]
    else:
        result = 1
    return result

In [25]:
comment_ticker_split_df['momentum_factor'] = comment_ticker_split_df[['momentum_factor', 'symbol']].apply(lambda x: extract_momentum(x.momentum_factor, x.symbol), axis=1)
comment_ticker_split_df = comment_ticker_split_df.drop_duplicates()

In [26]:
comment_ticker_split_df.head(10)

,comment_id,text,timestamp,type,momentum_factor,symbol
1,gpf8ky5,Honestly mods why didn't you make an actual GM...,1614698845,C,10.0,GME
6,gpf8vly,Let's just talk GME here,1614698980,C,10.0,GME
8,gpf8w2i,Holding RKT and GME. Suck on that Mods,1614698986,C,10.0,GME
9,gpf8wqh,RKT TO THE MOON!!! 🚀🚀🌚🌚,1614698995,C,10.0,🚀🚀🌚🌚
11,gpf8xou,I like GME,1614699007,C,10.0,GME
14,gpf8yoh,"This sub isn’t just a GME cesspool, jfc people",1614699021,C,10.0,GME
15,gpf8z1l,So apparently GME thread gets deleted in SECON...,1614699025,C,10.0,GME
19,gpf92y2,Very fishy... no zjz posts no Unpopular pinnn...,1614699076,C,10.0,Unpopular
19,gpf92y2,Very fishy... no zjz posts no Unpopular pinnn...,1614699076,C,1.0,GME
22,gpf93e3,![img](emote|wsb_emojis|bull) on CCIV / PLTR (...,1614699081,C,1.0,CCIV / PLTR (


In [27]:
post_ticker_split_df['momentum_factor'] = post_ticker_split_df[['momentum_factor', 'symbol']].apply(lambda x: extract_momentum(x.momentum_factor, x.symbol), axis=1)
post_ticker_split_df = post_ticker_split_df.drop_duplicates()

In [28]:
post_ticker_split_df.head(10)

,post_id,text,timestamp,type,momentum_factor,symbol
2,lw4i3o,"Gentlemen,\n\nNow that the yield bubble is pop...",1614698764,P,10.0,PLTR
3,lw4ihe,CVSI is on the move up! Buy low sell high! Hav...,1614698789,P,10.0,CVSI
4,lw4ilh,There is probably some rule against this threa...,1614698796,P,10.0,GME
8,lw4jqn,Mods surely know that no GME thread = spam in ...,1614698877,P,10.0,GME
9,lw4juw,CBD Life Sciences Inc. is up today. It's a C...,1614698887,P,10.0,CBD Life Sciences Inc.
9,lw4juw,CBD Life Sciences Inc. is up today. It's a C...,1614698887,P,1.0,Amazon
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,10.0,FDA
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,1.0,NPS
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,1.0,TAM
10,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,1.0,EV


### The symbol column can have both the company names and the ticker symbol, therefore we need to match both of them. The following queries are for this purpose

### This query is used to match tickers from the symbol column of both the dataframes

In [41]:
ticker_match_query_string = f"""
 SELECT
 companyid, symbol
 FROM
  "equity-atlas_v3"."shareclassinfo"
 LIMIT 100000
""" 

df_ticker_match = pd.DataFrame(dnalab.query(ticker_match_query_string))

In [42]:
df_ticker_match.head(10)

,companyid,symbol
0,0C000036UT,NAFT
1,0C000009C8,8AK
2,0C0000AZ8H,PAL
3,0C00000I43,COG
4,0C000006WM,BHPG34
5,0C00006D96,CIX
6,0C000014PJ,ARGEF
7,0C00000NCL,DZV2
8,0C00000QI4,0NBX
9,0C00000XF9,GANT


### This query is used to match actual company names from the symbol column of both the dataframes

In [45]:
company_match_query_string = f"""
 SELECT
 companyid, shortname AS symbol
 FROM
 "equity-atlas_v3"."companyinfo" 
 LIMIT 100000
"""

df_company_match = pd.DataFrame(dnalab.query(company_match_query_string))

In [46]:
df_company_match.head(10)

,companyid,symbol
0,0C000006U4,Bank of Commerce Hldgs
1,0C000006V2,Baxter Intl
2,0C000006VC,BCE
3,0C000006TA,Baker Hughes
4,0C000006T0,AZZ
5,0C000006TJ,Ballistic Recovery
6,0C000006WE,Best Buy Co
7,0C00000726,Bunge
8,0C0000BZIL,Luxbright
9,0C0000071J,Brown-Forman


In [50]:
comment_ticker_match_df = pd.merge(df_ticker_match, comment_ticker_split_df, on=['symbol'])
comment_company_match_df = pd.merge(df_company_match, comment_ticker_split_df, on=['symbol'])
final_comment_df = comment_ticker_match_df.append(comment_company_match_df)
final_comment_df.head(200)

,companyid,symbol,comment_id,text,timestamp,type,momentum_factor
0,0C00000UKH,GME,gpf8ky5,Honestly mods why didn't you make an actual GM...,1614698845,C,10.0
1,0C00000UKH,GME,gpf8vly,Let's just talk GME here,1614698980,C,10.0
2,0C00000UKH,GME,gpf8w2i,Holding RKT and GME. Suck on that Mods,1614698986,C,10.0
3,0C00000UKH,GME,gpf8xou,I like GME,1614699007,C,10.0
4,0C00000UKH,GME,gpf8yoh,"This sub isn’t just a GME cesspool, jfc people",1614699021,C,10.0
...,...,...,...,...,...,...,...
105,0C00000W3J,SLR,gpf9j57,Bearish imo. If SLR is re-instated to pre 3/20...,1614699286,C,10.0
106,0C00000IMZ,SLR,gpf9j57,Bearish imo. If SLR is re-instated to pre 3/20...,1614699286,C,10.0
107,0C000013S3,NAKD,gpf9iy5,Bot. Buying puts on NAKD. Thanks.\n\nReally ...,1614699283,C,10.0
0,0C000007ST,GameStop,gpf967y,"Sir, this is a GameStop dispensary.\n\n*I am a...",1614699117,C,10.0


In [51]:
post_ticker_match_df = pd.merge(df_ticker_match, post_ticker_split_df, on=['symbol'])
post_company_match_df = pd.merge(df_company_match, post_ticker_split_df, on=['symbol'])
final_post_df = post_ticker_match_df.append(post_company_match_df)
final_post_df.head(200)

,companyid,symbol,post_id,text,timestamp,type,momentum_factor
0,0C000009B9,FDA,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,10.0
1,0C000009B9,FDA,lw4nrn,**POSITION 3000 SHARES AT $10.15**\n\nI’ve be...,1614699179,P,10.0
2,0C000009B9,FDA,lw4rj7,\n\n**POSITION 3000 SHARES AT $10.15**\n\nI’v...,1614699443,P,10.0
3,0C000009B9,FDA,lw4k7e,\n\n*Processing img qkp7svjbvmk61...*\n\n I’v...,1614698913,P,10.0
4,0C000009B9,FDA,lw4nrn,**POSITION 3000 SHARES AT $10.15**\n\nI’ve be...,1614699179,P,10.0
...,...,...,...,...,...,...,...
153,0C00000SAL,KNOW,lw4rc5,"That hair, that smile, those glasses, that voi...",1614699428,P,10.0
154,0C00000BJN,IP,lw57p7,\n\n**POSITION 3000 SHARES AT $10.15**\n\nI’v...,1614700562,P,10.0
155,0C000014DD,SCAM,lw53jt,I have forex courses of profitable traders for...,1614700295,P,10.0
0,0C000008SF,RH,lw54kf,I'm a bit scared that RH will fuck people on t...,1614700354,P,10.0


### Save Extraction Files

In [52]:
final_comment_df.to_csv('extracted_comments.csv')
final_post_df.to_csv('extracted_posts.csv')

### Zip CSV Files and Submit